## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-24-02-16-09 +0000,bbc,"Trump will meet with Xi in South Korea, says W...",https://www.bbc.com/news/articles/cjw9vy19qvgo...
1,2025-10-24-02-13-29 +0000,nypost,Former special counsel Jack Smith asks to test...,https://nypost.com/2025/10/23/us-news/former-s...
2,2025-10-24-02-05-57 +0000,startribune,"Live: Justin Herbert, Chargers rolling over th...",https://www.startribune.com/minnesota-vikings-...
3,2025-10-24-02-01-37 +0000,bbc,NBA stars and mafia among dozens arrested in i...,https://www.bbc.com/news/articles/cvgmpp3x5x5o...
4,2025-10-24-01-46-57 +0000,nyt,Suspect in Palisades Fire Pleads Not Guilty to...,https://www.nytimes.com/2025/10/23/us/los-ange...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2384/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,61
116,new,20
157,off,14
48,up,13
707,sanctions,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
366,2025-10-23-03-10-21 +0000,nypost,Entire White House East Wing will be demolishe...,https://nypost.com/2025/10/22/us-news/entire-w...,143
132,2025-10-23-20-01-18 +0000,nypost,Here are the corporations and donors footing t...,https://nypost.com/2025/10/23/us-news/here-are...,122
122,2025-10-23-20-25-31 +0000,nyt,Trump’s Sanctions on Russian Oil Sector Ratche...,https://www.nytimes.com/2025/10/23/us/politics...,121
276,2025-10-23-11-45-10 +0000,bbc,Trump says Putin talks 'don't go anywhere' as ...,https://www.bbc.com/news/articles/cd6758pn6ylo...,120
253,2025-10-23-13-32-00 +0000,nyt,Trump Imposes Sanctions on Russian Oil Compani...,https://www.nytimes.com/2025/10/22/us/politics...,115


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
366,143,2025-10-23-03-10-21 +0000,nypost,Entire White House East Wing will be demolishe...,https://nypost.com/2025/10/22/us-news/entire-w...
122,60,2025-10-23-20-25-31 +0000,nyt,Trump’s Sanctions on Russian Oil Sector Ratche...,https://www.nytimes.com/2025/10/23/us/politics...
97,52,2025-10-23-21-24-00 +0000,wsj,Lawmakers failed to provide a last-minute repr...,https://www.wsj.com/politics/policy/air-traffi...
66,49,2025-10-23-22-21-54 +0000,nypost,How mafiosos and NBA stars used rigged card sh...,https://nypost.com/2025/10/23/us-news/how-mafi...
251,41,2025-10-23-13-34-07 +0000,nypost,"King Charles, Pope Leo pray together in rare m...",https://nypost.com/2025/10/23/world-news/king-...
207,41,2025-10-23-15-41-00 +0000,wsj,Honeywell Third-Quarter Aerospace Sales Jump A...,https://www.wsj.com/business/earnings/honeywel...
48,38,2025-10-23-23-29-06 +0000,nypost,Ford warns devasting plant fire will hurt prof...,https://nypost.com/2025/10/23/business/ford-wa...
161,37,2025-10-23-18-06-16 +0000,nypost,French museum was robbed of 2K gold and silver...,https://nypost.com/2025/10/23/world-news/a-sec...
211,36,2025-10-23-15-32-37 +0000,nypost,Zohran Mamdani’s dad allegedly tried to silenc...,https://nypost.com/2025/10/23/world-news/zohra...
361,34,2025-10-23-04-01-00 +0000,wsj,U.S. Widens Campaign Against Alleged Drug Boat...,https://www.wsj.com/world/americas/u-s-widens-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
